# Mesogens with NP | Temperature Ramps

## Temperature: 6.0 -> 6.2 | CPU Run

### Date: 12/09/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [1]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables.

press = 2.9;
t_0   = 8.0; 
t_1   = 7.0;
steps_ramp = 1.5e4;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(press) + "_ramp.gsd"

In [4]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree(r_buff = 3.5, check_period = 1);

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 0.5, tauP = 0.4, P = press);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(press) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(press) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(press) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.7.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/02/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1010 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [5]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9010
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 296188 / 311000 | TPS 18.7361 | ETA 00:13:10
Time 00:00:20 | Step 296401 / 311000 | TPS 20.9094 | ETA 00:11:38
Time 00:00:30 | Step 296620 / 311000 | TPS 21.8823 | ETA 00:10:57
Time 00:00:40 | Step 296816 / 311000 | TPS 19.5535 | ETA 00:12:05
Time 00:00:50 | Step 297008 / 311000 | TPS 19.1713 | ETA 00:12:09
Time 00:01:00 | Step 297204 / 311000 | TPS 19.5641 | ETA 00:11:45
Time 00:01:10 | Step 297392 / 311000 | TPS 18.1208 | ETA 00:12:30
Time 00:01:20 | Step 297580 / 311000 | TPS 18.7957 | ETA 00:11:53
Time 00:01:30 | Step 297788 / 311000 | TPS 20.7355 | ETA 00:10:37
Time 00:01:40 | Step 297969 / 311000 | TPS 18.0828 | ETA 00:12:00
Time 00:01:50 | Step 298165 / 311000 | TPS 19.5818 | ETA 00:10:55
Time 00:02:01 | Step 298367 / 3

In [6]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x7f2fbc8df8d0>),
             ('particles', <hoomd.data.particle_data at 0x7f2fbc80dac8>),
             ('number_density', 0.6028443219333094),
             ('bonds', <hoomd.data.bond_data at 0x7f2fbc80db38>),
             ('angles', <hoomd.data.angle_data at 0x7f2fbc80dba8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x7f2fbc80dc18>),
             ('impropers', <hoomd.data.dihedral_data at 0x7f2fbc80dc88>),
             ('constraints', <hoomd.data.constraint_data at 0x7f2fbc80dcf8>),
             ('pairs', <hoomd.data.bond_data at 0x7f2fbc80dd68>),
             ('timestep', 311000)])

In [7]:
#-----Get volume.

system.box.get_volume()

14945.815482022148